# Simulação de aplicação

O paradigma SSVEP é bastante utilizado em aplicações reais por ser fácil de evocar os sinais e posteriormente classificá-los.

Para processar uma aplicação SSVEP em tempo real é necessário calcular a máxima energia dos _targets_ em janelas ou _buffers_ suficientemente grandes. O tamanho da janela de dados é inversamente proporcional a taxa de amostragem do equipamente de EEG utilizado. Ou seja, quanto maior a taxa de amostragem, menor o tempo decessário para calcular a **energia deslizante do sinal**.

**Tarefa**: simule uma aplicação SSVEP criando um buffer do sinal no domínio do tempo em uma única dimensão. Se inicialmente a dimensão dos dados era `(100, 8192)`, agora será `(819200)`. Crie um buffer de 3 ou 4 segundos, informando a cada segundo qual está sendo a frequência evocada.

In [4]:
import numpy as np
import mne

# Definir parâmetros
fs = 8192  # Taxa de amostragem (Hz)
duration = 10  # Duração do sinal em segundos
buffer_size = 3  # Tamanho da janela em segundos (buffer de 3 segundos)
n_samples = fs * duration  # Número total de amostras
n_buffer_samples = fs * buffer_size  # Número de amostras por janela

# Gerar dados fictícios (Exemplo de sinal de EEG simulado)
# Usaremos uma combinação de frequências para simular um sinal de EEG com componentes SSVEP
time = np.arange(n_samples) / fs  # Tempo total do sinal
signal = np.sin(2 * np.pi * 10 * time) + 0.5 * np.sin(2 * np.pi * 15 * time)  # Frequências de 10Hz e 15Hz

# Adicionar ruído ao sinal para torná-lo mais realista
np.random.seed(42)
signal += 0.1 * np.random.randn(n_samples)

# Converter o sinal para um objeto Epochs do MNE (simulando que ele vem de um arquivo de EEG)
# Aqui estamos criando um único canal de EEG
info = mne.create_info(ch_names=['EEG'], sfreq=fs, ch_types=['eeg'])
raw_data = np.array([signal])
raw = mne.io.RawArray(raw_data, info)

# Função para processar cada buffer (janela) de dados
def process_buffer(buffer, fs, fmin=3, fmax=30):
    # Calcular a PSD usando o método multitaper
    psd, freqs = mne.time_frequency.psd_array_multitaper(buffer, sfreq=fs, fmin=fmin, fmax=fmax, verbose=False)
    
    # Extrair a PSD do buffer (o dado retornado é por canal e por frequência)
    psd_values = psd  # Para o primeiro (e único) canal
    
    # Encontrar a frequência com maior energia
    max_idx = np.argmax(psd_values)  # Índice da maior energia
    max_freq = freqs[max_idx]  # Frequência associada à maior energia
    
    return max_freq

# Simulação de um processo em tempo real com janela deslizante
for start_idx in range(0, len(signal) - n_buffer_samples, n_buffer_samples):
    end_idx = start_idx + n_buffer_samples
    buffer = signal[start_idx:end_idx]  # Extrair o buffer do sinal
    
    # Processar o buffer e calcular a frequência máxima
    max_freq = process_buffer(buffer, fs)
    
    # Imprimir a frequência evocada
    print(f"Tempo: {start_idx/fs:.2f} - Frequência Evocada: {max_freq:.2f} Hz")


Creating RawArray with float64 data, n_channels=1, n_times=81920
    Range : 0 ... 81919 =      0.000 ...    10.000 secs
Ready.
Tempo: 0.00 - Frequência Evocada: 10.00 Hz
Tempo: 3.00 - Frequência Evocada: 9.67 Hz
Tempo: 6.00 - Frequência Evocada: 9.67 Hz
